# Prévision de la consommation d'électricité en France.

### Abstract
Chaque jour, le gestionnaire de réseau RTE (Réseau de Transport de l'Electricité) ajuste la production de manière à fournir à chacun, à chaque instant, la quantité d'électricité nécessaire à sa consommation. Cet ajustement doit être particulièrement précis car les lois de la physiques imposent que la quantité d'électricité qui entre dans le système doit être égale à la quantité qui en sort.

D'autre part, du fait de la nature des facteurs de production, qui peuvent être intermittent ou au contraire ajustables (ex: il est possible d'allumer et d'éteindre une centrale à charbon sur demande), il est important de connaître à l'avance la demande en électricité de manière à choisir à l'avance les facteur de production à utiliser. 

Dans ce contexte il apparaît que prédire les consommation de l'énergie est un enjeu majeur. 

Toutefois, les outils de la statistiques permettent de construire des modèles capables de prédire la consommation de manière suffisamment précise pour éviter tant les courts-circuits que les pénuries. En effet, malgré l'incertitude, la consommation d'électricité semble être fortement influencée par différents facteur. 

Quels sont alors ces facteurs ? Comment les déterminer ? Comment batir un modèle de prédiction de la consommation sufisamment efficace ?

Afin de déterminer ceux-ci, nous allons faire appelle dans une première partie à l'étude des statistiques descriptives des bases de données fournies par RTE sur la consommation et la production. Dans un second temps, nous utiliserons nous construirons un modèle de régression linéaire afin de prédire la demande 


Pour réaliser notre projet, principalement les données publiques fournies par RTE 
Plan : 

0.[Acquisition et nettoyage des données](#Acquisition)

1.[Statistiques descriptives](#Statistiques)

2.[Modèle de Régression](#Modèle)

3.[Conclusion](#Conclusion)




In [72]:
!pip install geopandas

    ERROR: Command errored out with exit status 1:


  Using cached geopandas-0.8.1-py2.py3-none-any.whl (962 kB)
  Using cached pyproj-3.0.0.post1-cp38-cp38-win_amd64.whl (14.4 MB)
  Using cached Shapely-1.7.1-cp38-cp38-win_amd64.whl (1.0 MB)
  Using cached Fiona-1.8.18.tar.gz (1.3 MB)



     command: 'C:\Users\nicol\anaconda3\python.exe' -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\nicol\\AppData\\Local\\Temp\\pip-install-9z4p3ita\\fiona\\setup.py'"'"'; __file__='"'"'C:\\Users\\nicol\\AppData\\Local\\Temp\\pip-install-9z4p3ita\\fiona\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base 'C:\Users\nicol\AppData\Local\Temp\pip-pip-egg-info-qm_k4ewa'
         cwd: C:\Users\nicol\AppData\Local\Temp\pip-install-9z4p3ita\fiona\
    Complete output (1 lines):
    A GDAL API version must be specified. Provide a path to gdal-config using a GDAL_CONFIG environment variable or use a GDAL_VERSION environment variable.
    ----------------------------------------
ERROR: Command errored out with exit status 1: python setup.py egg_info Check the logs for full command output.


In [73]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import sklearn.metrics
import statsmodels.api as sm
import statsmodels.formula.api as smf

## 0)Acquisition et nettoyage des données 

Météo-France met à disposition des données météorologiques remontant jusqu'à 1996 et avec une fréquence d'enregistrement de 3h. Ces données sont téléchargeables en tables mensuelles à l'url https://public.opendatasoft.com/explore/dataset/donnees-synop-essentielles-omm/information/?refine.date=2020%2F12%2F09&fbclid=IwAR0RXR54Jve66dUu5yQqMc6hEwnGjedb6BcPQGI3Rmk0DcfdPIGCWN8fLNI .En concaténant les tables mensuelles pour les années 2018, 2019 et 2020 et en ne gardant que les variables que l'on souhaite étudier, on obtient une base de donnée synthétique, avec de quelques impuretés. Il est donc nécessaire de la nettoyer.


In [74]:
from numpy import nan
Gbase=pd.read_csv("https://raw.githubusercontent.com/yvan-belakebi/prediction-conso/contrib-yvan-bis/gbase")
basemod=Gbase.copy()
basemod=basemod.replace("mq",nan)
basemod=basemod.replace("t", nan)
basemod=basemod.dropna()
basemod["t"]=basemod["t"].astype(float)
basemod["date"]=basemod["date"].astype(str)
basemod["numer_sta"]=basemod["numer_sta"].astype(int)
basemod=basemod.rename(columns={'numer_sta':'ID'})
basemod.head(5)

C:\Users\nicol\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (1,2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Unnamed: 0,ID,date,t
0,0,7005,20180101000000,280.15
1,1,7015,20180101000000,281.05
2,2,7020,20180101000000,283.45
3,3,7027,20180101000000,280.55
4,4,7037,20180101000000,279.55


Nous avons donc la température mesurée par chaque station, à toutes les dates. L'objectif est de transformer cette température en heating degree days et cooling degree days. Heating degree days est la différence si elle est positive entre une température de référence, à partir de laquelle on considère que les ménages allument le chauffage, et la température mesurée. Cooling degree days est son pendant pour la climatisation. On va ensuite passer ces variables à l'échelle nationale en effectuant une moyenne pondérée par la population représentée par chaque station. Pour cela, on va attribuer à chaque station le département dans lequel elle se situe par reverse geocoding, et estimer la population représentée par une station à la population du département sur le nombre de stations dans le département.

In [75]:
import bs4
import urllib
import pandas as pd
import numpy as np
from urllib import request

url_popdep="https://fr.wikipedia.org/wiki/Liste_des_d%C3%A9partements_fran%C3%A7ais_class%C3%A9s_par_population_et_superficie"

request_text=request.urlopen(url_popdep).read()
page = bs4.BeautifulSoup(request_text, "lxml")
tableau_dep=page.find('table',{"class":"wikitable sortable alternance"})
table_body=tableau_dep.find('tbody')
rows=table_body.find_all('tr')

dico_dep = dict()
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols[1:]]
    if len(cols) > 0 :
        dico_dep[cols[0]] = cols[0:]

data_dep = pd.DataFrame.from_dict(dico_dep,orient='index')
#data_dep.to_csv("C:/Users/ADMIN/Documents/projet python 2A/data_dep")


for row in rows:
    cols = row.find_all('th')
    if len(cols) > 0 :
        cols = [ele.get_text(separator=' ').strip().title() for ele in cols]
        columns_dep = cols

import re

columns_années = [re.sub('\[ (\d+) \] ?', '', nom_col) for nom_col in columns_dep]
columns_dep=['Code','Département']+columns_années+['superficie','densité']

data_dep.columns = columns_dep[0:]
popdep=data_dep[['Code','Département','2017 ']]


def transfo(s):
    return re.sub("\D","",s)

station_dep=pd.read_csv("https://raw.githubusercontent.com/yvan-belakebi/prediction-conso/main/station_finale")
station_dep=pd.merge(basemod,station_dep)
popdep=popdep.rename(columns={'Code':'code_departement'})
popdep=popdep.astype({'2017 ':'str'}, errors='ignore')
popdep['2017 ']=popdep['2017 '].apply(transfo)
popdep['test']=popdep['2017 '].str.isdigit()
popdep=popdep.query('test')

popdep=popdep.astype({'2017 ':'int'})

popdep['weight']=popdep['2017 ']/popdep['2017 '].sum()
popdep.head(5)

,code_departement,Département,2017,test,weight
59,59,Nord,2604361,True,0.039149
75,75,Paris,2187526,True,0.032883
13,13,Bouches-du-Rhône,2024162,True,0.030427
69,69,Circonscription départementale du Rhône,1843319,True,0.027709
93,93,Seine-Saint-Denis,1623111,True,0.024399


In [76]:
station_pop=pd.merge(station_dep,popdep, on='code_departement')
station_pop['code_departement']=station_pop['code_departement'].replace("2A","2")
station_pop['code_departement']=station_pop['code_departement'].replace("2B","2")
station_pop=station_pop.astype({"code_departement":'int'})


tbasse=289.15  #16°C, température à partir de laquelle on chauffe
thaute=301.15  #28°C, température à partir de laquelle on climatise

station_pop['hdd']=np.maximum(0, tbasse-station_pop['t'])
station_pop['cdd']=np.maximum(0, station_pop['t']-thaute)

station_pop.head(5)

,Unnamed: 0,ID,date,t,code_departement,Département,2017,test,weight,hdd,cdd
0,0,7005,20180101000000,280.15,80,Somme,572443,True,0.008605,9.0,0.0
1,160686,7299,20181226150000,273.45,68,Haut-Rhin,764030,True,0.011485,15.7,0.0
2,279090,7627,20190902150000,293.85,9,Ariège,153153,True,0.002302,0.0,0.0


Il suffit ensuite de réunir les bases station_pop et consommation avec un merge pour obtenir une base prête pour la régression

## 1) Statistiques descriptives 

Dans un premier temps, on s'intéresse à la base de donnée sur la consommation d'électricité à l'échelle de la France durant l'année 2018. 

In [ ]:
cons_fr_2018 = pd.read_csv('https://raw.githubusercontent.com/yvan-belakebi/prediction-conso/main/cons2018.csv',encoding='ISO-8859-1',error_bad_lines=False,sep = ';')
# On supprime les lignes avec des valeurs manquantes
cons_fr_2018 = cons_fr_2018.drop(['Nature'],axis=1)
cons_fr_2018.dropna(inplace=True)
# On restreint la base de données aux variables qui nous intéresse.
cons_fr_2018 = cons_fr_2018.iloc[:,0:13]
cons_fr_2018.head(10)
# On crée une variable correspondant aux mois.
cons_fr_2018['Date'] = pd.to_datetime(cons_fr_2018['Date'],format="%d/%m/%Y")
cons_fr_2018['Mois'] = cons_fr_2018['Date'].astype(str).str[5:7]

On va s'intéresser au profil de production des énergies suivantes à la fois au niveau mensuel et au niveau journalier pour les énergies renouvelables. En effent, il n'est pas très intéressant de comparer les renouvelables avec les autres sources de production comme les centrales à charbon car on peut allumer et éteindre une centrale à charbon tandis que les renouvelables produisent tout le temps.


In [ ]:
moyenne_j_eol = cons_fr_2018.groupby('Heures')['Eolien'].mean().plot()
moyenne_j_sol = cons_fr_2018.groupby('Heures')['Solaire'].mean().plot()
moyenne_j_gaz = cons_fr_2018.groupby('Heures')['Gaz'].mean().plot()
plt.title('Profil de production journalier des différentes énergies')
plt.ylabel('Production en MW')
plt.legend()
plt.show()



In [ ]:
moyenne_a_eol = cons_fr_2018.groupby('Mois')['Eolien'].mean().plot()
moyenne_a_sol = cons_fr_2018.groupby('Mois')['Solaire'].mean().plot()
moyenne_a_gaz = cons_fr_2018.groupby('Mois')['Gaz'].mean().plot()
plt.title('Profil de production annuel des différentes énergies')
plt.ylabel('Production en MW')
plt.legend()
plt.show()

In [ ]:
On voit bien ici que les différentes sources de production ont des profil différents. 

Maintenant, on veut comprendre plus en détail le profil journalier moyen de la consommation. Nous cherchons donc à faire un graphique permettant de d'avoir une idée des variation intrajournalières de la consommation.


In [ ]:
moyenne_cons = cons_fr_2018.groupby('Heures')['Consommation'].mean()
plt.title('Variations intrajournalières de la consommation en France en 2018')
plt.ylabel('Consommation en MW')
moyenne_cons.plot()

Même idée mais pour la consommation annuelles.


In [ ]:
moyenne_cons = cons_fr_2018.groupby('Mois')['Consommation'].mean()
plt.title('Variations intramensuelles de la consommation en France en 2018')
plt.ylabel('Consommation en MW')
moyenne_cons.plot()

In [ ]:
# Création de la variable mois 
# On passe la date au format date 



A la lumière des deux graphique que nous venons de construire, il apparaît qu'il y a deux effet qui jouent sur la consommation d'électricité en France. On distingue donc un effet journalier : les gens ont tendance à consommer beaucoup plus d'électricité le jour que la nuit. Plus précisément, la courbe est creuse entre 00h00 et 05h00 tandis qu'elle est à son pic entre 10h00 et 20h00. 

Le second effet est mensuel. La consommation est plus élevée en hiver qu'en été. Notre intuition là-dessus est que les gens chauffe beaucoup avec des chauffage électrique lorsqu'il fait froid tandis qu'en été, les gens ont moins tendance à utiliser le climatiseur. 


## 2) Modèle de régression linéaire 

Afin de constuire un modèle de régression linéaire permettant de prédire au mieux la consommation, nous sommes partie de notre intuition. En effet, en plus de leur et de la période de l'année, la température semble pourrait jouer une rôle important sur la demande. Ainsi on crée deux variables qui renvoient aux températures suffisamment haute pour déclencher l'utilisation du climatiseur et aux variables suffisamment basses pour déclencher l'utilisation du radiateur. 




In [ ]:
reg_table = pd.read_csv('https://raw.githubusercontent.com/yvan-belakebi/prediction-conso/contrib-yvan-bis/base_synth%C3%A9')
# Création de la variable 'Heure'
reg_table['Heures'] = reg_table['date'].astype(str).str[8:10].astype(str)
# On crée les dummy variable qui nous intéressent
reg_table['HB_s1'] = reg_table['Heures'] == '03'
reg_table['HB_s2'] = reg_table['Heures'] == '00'
reg_table['HB_s3'] = reg_table['Heures'] == '06'
reg_table['HB'] = reg_table['HB_s1'].astype(int) + reg_table['HB_s2'].astype(int) + reg_table['HB_s3'].astype(int)

reg_table['HH_s1'] = reg_table['Heures'] == '09'
reg_table['HH_s2'] = reg_table['Heures'] == '12'
reg_table['HH_s3'] = reg_table['Heures'] == '15'
reg_table['HH_s4'] = reg_table['Heures'] == '18'
reg_table['HH_s5'] = reg_table['Heures'] == '21'
reg_table['HH'] = reg_table['HH_s1'].astype(int) + reg_table['HH_s2'].astype(int)+ reg_table['HH_s3'].astype(int) + reg_table['HH_s4'].astype(int)+ reg_table['HH_s5']
# On entraîne notre modèle
x_train, x_test, y_train, y_test = train_test_split(reg_table[['cdd_pond','hdd_pond','HH','HB']],
                                                    reg_table[['Consommation']],
                                                    test_size=0.2)

ols = LinearRegression()
ols.fit(x_train, y_train)
ols.score(x_test, y_test)

In [ ]:
ols.intercept_, ols.coef_

In [ ]:
ols_model = sm.OLS(reg_table["Consommation"].astype(float), reg_table[['cdd_pond','hdd_pond','HH','HB']].astype(float))

In [ ]:
results_sm = ols_model.fit()
results_sm.summary()

In [ ]:
plt.scatter(y_test, y_test - ols.predict(x_test))


Finalement, notre modèle de régression linéaire peut donc s'écrire : 

$ Consommation = (-208)\times cdd_{pond} + 2244\times hdd_{pond} + 45770\times HH + 40640\times HB + \epsilon $


##  Interprétation économétrique du modèle
Si on prend on compte les résultats de statmodel, on remarque qu'il n'ya pas d'intercept. Toutefois ils permettent d'entrevoir certaines tendance. Comme l'avions supposé, la variable cdd_pond qui correspond à la variation de consommation causée par des température élevée est la moins pertinente du modèle. En effet, on ne peut pas rejeter l'hypothèse que cdd_pond soit nul. C'est l'intuition que nous avions eu en regardant le graphique de la variation de la consommation au cours d'une année. 
 
On remarque que les variables HH et HB sont les plus significatives du modèle. Elle correspondent à ce que l'on pourrait appeler des heures pleines et des heures creuses. Si on prend en compte la régression donnée par sklearn, la variablle corrélée très positivement à la consommation tandis que la variable HB ( heures creuses ) est corrélée négativement. 

Le Scatter plot montre que les résidus de notre régression sont plus élevées lorsque les valeurs observées sont plus élevées. Cela contredit l'hypothèse d'homoscedascticité. 

Enfin la valeur du R² est de 0.626 ce qui n'est pas suffisant comme prédiction. 


## 3) Conclusion

Notre régression explique une partie de la consommation mais le R² est trop faible pour que les préctions soient viables. 
Dans le but d'améliorer notre modèle, nous voyons les pistes d'amélioration suivante : 

- Ajouter des variables calendaires permettant de faire la distinction entre les jours fériés et les jours ouvrés. Il nous semble que la demande d'électricité varie également beaucoup selon si on est en semaine ou le week-end. 

- Optimiser les températures seuil de manière à avoir le meilleur R². 
